In [ ]:
import requests
import pandas as pd

In [ ]:
col_description = pd.read_csv('./HomeCredit_columns_description.csv', encoding='ISO-8859-1', index_col=0)



In [50]:
col_description[col_description['Row']=='SK_ID_CURR']['Description'][1]

'ID of loan in our sample'

In [ ]:
def prepare_data_for_model(df, dataset_version:str):

    if dataset_version == '2.0':
        df.columns = df.columns.str.replace('[^\w\s]','')
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        df.drop(columns='level_0', inplace=True)
        columns_X = [col for col in df.columns if col not in not_usable_col]
        df = df[columns_X]
    return df

In [ ]:
train_data = pd.read_csv("./sampled_train_data.csv", index_col=0)

In [ ]:
X1 = train_data[train_data['SK_ID_CURR'] == 30742]

In [ ]:
X1 = prepare_data_for_model(X1, '2.0')
X1

In [ ]:
#get the model dataset version:
response = requests.get('http://127.0.0.1:5001/threshold')
print("Status code: ", response.status_code)
print("Response content: ", response.json())

In [ ]:
#Get global_importance png
response = requests.get('http://127.0.0.1:5001/global_importance')
if response.status_code == 200:
    with open("shap_summary.png", "wb") as f:
        f.write(response.content)

In [ ]:
#Get the predict_proba out put is in form of :
#[[0.0, 0.14061783578431664, 0.8593821642156834]] class prediction, %of class 0, % of class1

data = X1.to_json(orient='records')
headers = {'Content-type': 'application/json'}
response = requests.post('http://127.0.0.1:5001/predict', data=data, headers=headers)

print("Response content: ", response.json())

In [ ]:
#Get the force plot as an html file
data = X1.to_json(orient='records')
headers = {'Content-type': 'application/json'}
response = requests.post('http://127.0.0.1:5001/get_local_importance', data=data, headers=headers)

if response.status_code == 200:
    with open("local_importance.html", "wb") as f:
        f.write(response.content)
